In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
df = pd.read_csv("cpu.csv")

In [3]:
df.sample(5)

,vendor name,model name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
77,harris,700,300,384,12000,6,1,24,53,50
70,hp,3000/64,75,2000,8000,8,3,24,62,47
179,prime,50-750-ii,160,1000,8000,16,1,14,60,43
201,sperry,80/4,180,512,4000,0,1,3,14,24
143,nas,as/6130,75,4000,16000,16,1,6,66,86


In [4]:
df.columns

Index(['vendor name', 'model name', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN',
       'CHMAX', 'PRP', 'ERP'],
      dtype='object')

In [5]:
print("vendor name : ", np.unique(df['vendor name'].values))
print("model name : ", np.unique(df['model name'].values))
print("MYCT : ", np.unique(df['MYCT'].values))
print("MMIN : ", np.unique(df['MMIN'].values))
print("MMAX : ", np.unique(df['MMAX'].values))
print("CACH : ", np.unique(df['CACH'].values))
print("CHMIN : ", np.unique(df['CHMIN'].values))
print("CHMAX : ", np.unique(df['CHMAX'].values))
print("PRP : ", np.unique(df['PRP'].values))
print("ERP : ", np.unique(df['ERP'].values))

vendor name :  ['adviser' 'amdahl' 'apollo' 'basf' 'bti' 'burroughs' 'c.r.d' 'cambex'
 'cdc' 'dec' 'dg' 'formation' 'four-phase' 'gould' 'harris' 'honeywell'
 'hp' 'ibm' 'ipl' 'magnuson' 'microdata' 'nas' 'ncr' 'nixdorf'
 'perkin-elmer' 'prime' 'siemens' 'sperry' 'sratus' 'wang']
model name :  ['100' '1100/61-h1' '1100/81' '1100/82' '1100/83' '1100/84' '1100/93'
 '1100/94' '1636-1' '1636-10' '1641-1' '1641-11' '1651-1' '2000/260' '300'
 '3000/30' '3000/40' '3000/44' '3000/48' '3000/64' '3000/88' '3000/iii'
 '3033:s' '3033:u' '3081' '3081:d' '3083:b' '3083:e' '32' '32/60' '3205'
 '3210' '3230' '370/125-2' '370/148' '370/158-3' '38/3' '38/4' '38/5'
 '38/7' '38/8' '4321' '4331-1' '4331-11' '4331-2' '4341' '4341-1'
 '4341-10' '4341-11' '4341-12' '4341-2' '4341-9' '4361-4' '4361-5'
 '4381-1' '4381-2' '4436' '4443' '4445' '4446' '4460' '4480' '470v/7'
 '470v/7a' '470v/7b' '470v/7c' '470v/b' '50-2250' '50-250-ii' '50-550-ii'
 '50-750-ii' '50-850-ii' '500' '5000' '580-5840' '580-5850' '580-586

In [6]:
df.drop(['model name', 'vendor name'], axis=1, inplace=True) # keep ordinal values only

In [7]:
df.sample(5)

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
196,50,2000,32000,112,52,104,307,341
122,1500,768,1000,0,0,0,12,18
117,225,1000,4000,2,3,6,24,26
177,160,512,2000,2,3,8,32,20
47,810,1000,5000,0,1,1,20,28


In [8]:
df['PRP'] = df['PRP'].astype(int)

In [9]:
df.loc[(df['PRP'] >= 0) & (df['PRP'] <= 20),'PRP'] = -1
df.loc[(df['PRP'] >= 21) & (df['PRP'] <= 100),'PRP'] = -2
df.loc[(df['PRP'] >= 101) & (df['PRP'] <= 200),'PRP'] = -3
df.loc[(df['PRP'] >= 201) & (df['PRP'] <= 300),'PRP'] = -4
df.loc[(df['PRP'] >= 301) & (df['PRP'] <= 400),'PRP'] = -5
df.loc[(df['PRP'] >= 401)& (df['PRP'] <= 500),'PRP'] = -6
df.loc[(df['PRP'] >= 501) & (df['PRP'] <= 600),'PRP'] = -7
df.loc[df['PRP'] >= 601,'PRP'] = -8

In [10]:
df.loc[df['PRP'] == -1, 'PRP'] = 1
df.loc[df['PRP'] == -2, 'PRP'] = 2
df.loc[df['PRP'] == -3, 'PRP'] = 3
df.loc[df['PRP'] == -4, 'PRP'] = 4
df.loc[df['PRP'] == -5, 'PRP'] = 5
df.loc[df['PRP'] == -6, 'PRP'] = 6
df.loc[df['PRP'] == -7, 'PRP'] = 7
df.loc[df['PRP'] == -8, 'PRP'] = 8

In [11]:
df = df[['PRP', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'ERP']]

In [12]:
df['PRP'] = df['PRP'].astype(int)

In [13]:
df.sample(3)

,PRP,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,ERP
106,2,400,4000,8000,0,1,1,45
155,5,38,8000,32000,64,8,24,267
184,2,105,2000,6000,16,6,16,41


In [14]:
vectors = df.values
dataset = LabeledSet(7)
dataset.addExamples(vectors[:,1:], vectors[:,0:1])

In [15]:
NMI1(dataset)

0.0012881854987118145

In [21]:
folds = get_folds(dataset, 2)

In [32]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

In [26]:
# Rank Shannon discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5308608058608059
average depth :  10.5
average number of leaves :  23.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9053021978021978
average number of pairs used for ratio computing :  5.0


In [27]:
# Shannon entropy 

avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(sdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.530952380952381
average depth :  9.0
average number of leaves :  18.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9425824175824176
average number of pairs used for ratio computing :  4.0


In [29]:
# Pessimistic rank discrimination measure

avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5550366300366301
average depth :  11.0
average number of leaves :  25.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8728296703296704
average number of pairs used for ratio computing :  5.0


In [30]:
# Rank Gini discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5213369963369964
average depth :  10.0
average number of leaves :  25.0
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8946428571428571
average number of pairs used for ratio computing :  6.0


In [33]:
# Gini discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5500915750915751
average depth :  8.5
average number of leaves :  19.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9090201465201466
average number of pairs used for ratio computing :  5.0


In [35]:
# H_M
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.545467032967033
average depth :  14.5
average number of leaves :  31.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8494276556776557
average number of pairs used for ratio computing :  6.0


In [36]:
# H_Q
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(2):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 2):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/2)
avg_leaves = avg_leaves * (1.0/2)
avg_depth = avg_depth * (1.0/2)
avg_ratio = avg_ratio * (1.0/2)
avg_pairs = avg_pairs * (1.0/2)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5403846153846154
average depth :  11.0
average number of leaves :  28.0
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.6313553113553114
average number of pairs used for ratio computing :  8.0
